# [Outbrain Clicks](https://www.kaggle.com/c/outbrain-click-prediction/)
Notes:
- [Sorting BTB](https://www.kaggle.com/clustifier/outbrain-click-prediction/btb-0-63/code)

In [10]:
import pandas as pd
import numpy as np 

data = pd.read_csv("data/clicks_train.csv", nrows=10000000) # real length 87141731
#test = pd.read_csv("data/clicks_test.csv", nrows=5000000) # real length 32225162
from sklearn.cross_validation import train_test_split
train = data[:9000000]
test = data[9000000:]

In [11]:
cnt = train[train.clicked==1].ad_id.value_counts()
cntall = train.ad_id.value_counts()

In [12]:
def get_prob(k):
    if k not in cnt:
        return 0
    return cnt[k]/float(cntall[k])

def srt(x):
    ad_ids = map(int, x.split())
    ad_ids = sorted(ad_ids, key=get_prob, reverse=True)
    return " ".join(map(str,ad_ids))

In [13]:
test['predict_click'] = test.ad_id.apply(lambda x: get_prob(x) > 0.5)
#subm['ad_id'] = subm.ad_id.apply(lambda x: srt(x))

/Users/jgzuke/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [30]:
# return true_positive, false_positive, true_negative, false_negative
def score(data):
    
    predict_clicked = data.predict_click
    clicked = data.clicked
    true_positive = data[predict_clicked & clicked]
    false_positive = data[predict_clicked & np.invert(clicked)]
    false_negative = data[np.invert(predict_clicked) & clicked]
    true_negative = data[np.invert(predict_clicked) & np.invert(clicked)]
    
    scores = (len(true_positive), len(false_positive), len(true_negative), len(false_negative))
    
    print ('f1: ' + str(scores[0] / (scores[0] + scores[1] + scores[3])))
    print (scores)

In [31]:
score(test.drop(['display_id', 'ad_id'], axis=1))

f1: 0.08258184034593474
(17207, 16089, 791637, 175067)
